In [111]:
import pymysql

In [112]:
import pandas as pd
from datetime import datetime

#step 1 : Loading the data 
df = pd.read_csv(r'C:\Users\surya\Downloads\traffic_stops - traffic_stops_with_vehicle_number.csv')
df.head()

C:\Users\surya\AppData\Local\Temp\ipykernel_10808\3442131631.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\surya\Downloads\traffic_stops - traffic_stops_with_vehicle_number.csv')


,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,0:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,0:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,0:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,0:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,0:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305


In [ ]:
df['stop_duration'].unique()

In [113]:
duration_mapping = {
    '0-15 Min': 7.5,
    '16-30 Min': 22.5,
    '30+ Min': 30
}
df['timing'] = df['stop_duration'].map(duration_mapping)

In [114]:
# step 2: data cleaning 
#Dropping columns with all missing values

df.dropna(axis=1, how = 'all' , inplace = True )   # here axis=1 means drop columns and how = 'all' means drop the column only if all values are NaN.

# Fill NaNs with Suitable values
df.fillna({
    'driver_age' : df['driver_age'].median(),
    'search_type':'None',
    'stop_duration':'Unknown',
    'violatio':'Unknown',
    'stop_outcome':'Unknown',
},inplace=True)

In [115]:
df.duplicated().sum()

np.int64(0)

In [ ]:
df.info()

In [116]:
#Convert Date and time to timestamp 
df['timestamp']=pd.to_datetime(df['stop_date']+' '+df['stop_time'])

df['stop_date'] = pd.to_datetime(df['stop_date'], format="%Y-%m-%d")
df['stop_time'] = pd.to_datetime(df['stop_time'], format="%H:%M:%S")


In [ ]:
#Connecting to mysql

conn = pymysql.connect(
    host="localhost",
    user="root",
    password="1234"
)

cursor = conn.cursor()
cursor.execute("create database SecureCheck")   #Creating a DB named SecureCheck

In [119]:
cursor.execute("Use securecheck")  #Using the Securecheck DB


0

In [ ]:
###cursor.execute('DROP TABLE IF EXISTS police_logs')


In [120]:
cursor.execute("""CREATE TABLE IF NOT EXISTS police_logs(
   id int AUTO_INCREMENT PRIMARY KEY , 
   stop_date DATE , 
   stop_time TIME ,
   country_name varchar(20), 
   driver_gender varchar(10),
   driver_age int ,
   driver_race varchar(20),
   violation varchar(20),
   search_conducted BOOLEAN,
   search_type varchar(20),
   stop_outcome varchar(20),
   is_arrested BOOLEAN ,
   stop_duration varchar(50) , 
   drugs_related_stop BOOLEAN , 
   vehicle_number varchar(50),
   timestamp DATETIME ,
   timing float
)""")

0

In [121]:
df.dtypes

stop_date             datetime64[ns]
stop_time             datetime64[ns]
country_name                  object
driver_gender                 object
driver_age_raw                 int64
driver_age                     int64
driver_race                   object
violation_raw                 object
violation                     object
search_conducted                bool
search_type                   object
stop_outcome                  object
is_arrested                     bool
stop_duration                 object
drugs_related_stop              bool
vehicle_number                object
timing                       float64
timestamp             datetime64[ns]
dtype: object

In [ ]:
df.isnull().sum().sum()

In [ ]:
df.columns

In [125]:
#Filter and rename columns to match SQL table
df_filtered = df[['stop_date', 'stop_time', 'country_name', 'driver_gender',
                  'driver_age', 'driver_race', 'violation', 'search_conducted',
                  'search_type', 'stop_outcome', 'is_arrested', 'stop_duration',
                  'drugs_related_stop','vehicle_number','timing','timestamp']]


In [127]:
records = [tuple(x) for x in df_filtered.itertuples(index=False, name=None)]

cursor.executemany(insert_query, records)
conn.commit()

print("✅ Data inserted successfully!")


TypeError: not all arguments converted during string formatting

In [128]:
records = [tuple(x) for x in df_filtered.itertuples(index=False, name=None)]

records

[(Timestamp('2020-01-01 00:00:00'),
  Timestamp('1900-01-01 00:00:00'),
  'Canada',
  'M',
  19,
  'Asian',
  'Speeding',
  True,
  'Vehicle Search',
  'Ticket',
  True,
  '16-30 Min',
  True,
  'UP76DY3473',
  22.5,
  Timestamp('2020-01-01 00:00:00')),
 (Timestamp('2020-01-01 00:00:00'),
  Timestamp('1900-01-01 00:01:00'),
  'India',
  'M',
  58,
  'Other',
  'Other',
  False,
  'Vehicle Search',
  'Arrest',
  True,
  '16-30 Min',
  True,
  'RJ83PZ4441',
  22.5,
  Timestamp('2020-01-01 00:01:00')),
 (Timestamp('2020-01-01 00:00:00'),
  Timestamp('1900-01-01 00:02:00'),
  'USA',
  'M',
  76,
  'Black',
  'Speeding',
  False,
  'Frisk',
  'Ticket',
  True,
  '16-30 Min',
  True,
  'RJ32OM7264',
  22.5,
  Timestamp('2020-01-01 00:02:00')),
 (Timestamp('2020-01-01 00:00:00'),
  Timestamp('1900-01-01 00:03:00'),
  'Canada',
  'M',
  76,
  'Black',
  'DUI',
  True,
  'Frisk',
  'Warning',
  False,
  '0-15 Min',
  True,
  'RJ76TI3807',
  7.5,
  Timestamp('2020-01-01 00:03:00')),
 (Timestamp(

In [137]:
# Convert to datetime first
df['stop_time2'] = pd.to_datetime(df['stop_time'])

# Extract time and format it using strftime
df['stop_time2'] = df['stop_time2'].dt.time

In [138]:
df.dtypes

stop_date             datetime64[ns]
stop_time             datetime64[ns]
country_name                  object
driver_gender                 object
driver_age_raw                 int64
driver_age                     int64
driver_race                   object
violation_raw                 object
violation                     object
search_conducted                bool
search_type                   object
stop_outcome                  object
is_arrested                     bool
stop_duration                 object
drugs_related_stop              bool
vehicle_number                object
timing                       float64
timestamp             datetime64[ns]
stop_time2                    object
dtype: object